# Basic application

This notebook shows you how to use the *basic* Vitis network example that can be generated from this repository.
The design provides network connectivity using User Datagram Protocol (UDP) as the transport protocol.

The assumptions made is this notebook are:
1. You have an Alveo card configured with one of the supported shells
1. You have generated the xclbin file for the *basic* example
1. You have a 100 GbE capable NIC
1. Both the Alveo card and the 100 GbE capable NIC are in the same host
1. Alveo card is connected to the NIC, either directly or with a any network equipment, using any of the interfaces.

Let's have a look at the *basic* design.

There are 4 Kernels:
* CMAC: provides the translation between physical signals to AXI4-Stream interface
* Network layer: provides a bridge between raw Ethernet packets and the application using UDP as transport layer
    * ARP provides translation between MAC and IP addresses
    * ICMP provides ping capabilities
    * The UDP module has a 16-entry table with socket information that needs to be filled in before running
* krnl_mm2s: reads data from memory and packetize it setting tdest and tlast appropriately
* krnl_s2mm: read data from the stream and copy it to memory

![](../img/udp_network_basic.png)

In the followings cells we will:
* Import the necessary pynq and python packages, define current device and xclbin file
* Explore kernels in the design
* Check physical link
* Change Alveo card IP address and ping it
* Configure socket table and populate it to the UDP module
* Create UDP socket in the host
* Allocate Alveo buffers
* Move data from the HOST through the network (NIC) to the Alveo card
* Move data from the HOST through the Alveo card to the network (NIC)
* Free resources

In [ ]:
#Uncomment the next line and run to reset the FPGA if it is not taking programming or otherwise misbehaving
#!xbutil reset --device 0000:01:00.1 --force

## Import packages and program FPGA
In this section we need to import the `pynq` and python packages that will be used in the rest of this notebook. We also import the `vnx_utils.py` file with helper functions to set up the vnx examples.

In [ ]:
import pynq
import numpy as np
from _thread import *
import threading 
import socket
from vnx_utils import *

We also need to define the current device, only if there is more than one Alveo card on the host. First let's check how many devices are available.

In [ ]:
for i in range(len(pynq.Device.devices)):
    print("{}) {}".format(i, pynq.Device.devices[i].name))

* If there are more than one Alveo card available, you should pass the `device` argument to the `pynq.Overlay` class.
* The xclbin variable should point to the `xclbin` file for the *basic* example

In [ ]:
currentDevice = pynq.Device.devices[0]
xclbin = '../basic.intf0.xilinx_u280_gen3x16_xdma_1_202211_1/vnx_basic_if0.xclbin'
ol = pynq.Overlay(xclbin,device=currentDevice)

## Explore kernels in the design
This design was built for both interfaces. Therefore, we will see each kernel repeated twice. One for the interface 0 and another for the interface 1

In [ ]:
ol.ip_dict

## Check physical link
After the dynamic region of the Alveo card is programmed with the basic example we can start interacting with the design.

Let's check if the Alveo card has detected link with the network equipment. To do so, we will use one of the helper functions `link_status`

In [ ]:
print("Link interface 0 {}".format(ol.cmac_0.link_status()))

## Change Alveo card IP address and ping it
By defaul the Alveo IP address is `192.168.0.5` and MAC address is `00:0A:35:02:9D:E5`. Let's change it to `192.168.100.2`, in this example we are using interface 0

After the IP address is changed, we can ping the Alveo card. The first attempts will fail but the remaining should work.

**Make sure you have configured the IP address of the 100 GbE capable NIC to be in the same subnetwork as the Alveo card**

In [ ]:
alveo_ipaddr = '192.168.100.2'
print(ol.networklayer_0.set_ip_address(alveo_ipaddr, debug=True))

* Check 100 GbE capable NIC configuration

In [ ]:
!ip addr show ens3f0np0

* Run ping to get ARP set up
Some of the attempts will fail but the remaining should work.

In [ ]:
!ping -c 5 $alveo_ipaddr -I ens3f0np0

## Configure socket table and populate it to the UDP module

In this section we will configure the socket table in software and populate it to the UDP module in the Alveo card. 
1. Define a couple of connections 
1. The socket table is populated to the UDP module in the Alveo card using the `.populate_socket_table()` helper function

In [ ]:
sw_ip = '192.168.100.1'
ol.networklayer_0.sockets[0] = (sw_ip, 50446, 60133, True)
ol.networklayer_0.sockets[1] = (sw_ip, 38746, 62781, True)

ol.networklayer_0.populate_socket_table(debug=True)

## Create UDP socket in the host
In this part we will open an UDP socket in the host (software) to be able to communicate with the Alveo card through the network.

We will use the python socket API to do so, the socket will be binded to the port `38746`

In [ ]:
SW_PORT = ol.networklayer_0.sockets[1]['theirPort']
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) # UDP
sock.bind(('', SW_PORT))

## Allocate Alveo buffers

We need to allocate the buffers on the global memory of the Alveo card to be able to pull and push data from them. We also will initialize the the sending buffer, `mm2s_buf`, with random data. This random data will be sent later to the network.

1. Define alias for the application kernlels (`mm2s` and `s2mm`)
1. Define size and shape of the buffers
1. Allocate the buffers
1. Initialize sending buffer with random data

In [ ]:
mm2s = ol.krnl_mm2s_0
s2mm = ol.krnl_s2mm_0

size = 1408 * 100
shape = (size,1)

if hasattr(ol, 'HBM0'):
    mm2s_buf = pynq.allocate(shape, dtype=np.uint8, target=ol.HBM0)
    s2mm_buf = pynq.allocate(shape, dtype=np.uint8, target=ol.HBM0)
else:
    mm2s_buf = pynq.allocate(shape, dtype=np.uint8, target=ol.bank1)
    s2mm_buf = pynq.allocate(shape, dtype=np.uint8, target=ol.bank1)

mm2s_buf[:] = np.random.randint(low=0, high=((2**8)-1), size=shape, dtype=np.uint8)

## Move data from the HOST through the network (NIC) to the Alveo card

In this section we will write data to the socket, which will send such data from the host to the Alveo card using the network.

Start streaming to memory mapped kernel, we need to specify how much data to expect from the network.

In [ ]:
s2mm_wh = s2mm.start(s2mm_buf,size)

Initialize a buffer with random data, define the packet size and compute how many packets we need to send to transmit the whole buffer to the network. Write the data from the buffer into the socket in `BYTES_PER_PACKET` chunks.

In [ ]:
udp_message_global = np.random.randint(low=0, high=((2**8)-1), size=shape, dtype=np.uint8)
BYTES_PER_PACKET = 1408
num_pkts = size//BYTES_PER_PACKET
alveo_port = ol.networklayer_0.sockets[1]['myPort']
for m in range(num_pkts):
    udp_message_local = udp_message_global[(m * BYTES_PER_PACKET) : \
                        ((m * BYTES_PER_PACKET) + BYTES_PER_PACKET)]
    sock.sendto(udp_message_local, (alveo_ipaddr, alveo_port))

* Wait for the s2mm kernel to receive all the data
* Move data from global memory to HOST memory
* Compare what was sent against what was received and print out result

In [ ]:
s2mm_wh.wait()
s2mm_buf.sync_from_device()

msg = "SUCCESS!" if np.array_equal(udp_message_global, s2mm_buf) else "FAILURE!"
print("Host sending data through the network and the host getting data from kernel \
was a: {}. Total data transmitted {:,} bytes to {}" .format(msg,size,(alveo_ipaddr, alveo_port)))

## Move data from the HOST through the Alveo card to the network (NIC)

We need to create a new thread to read data from the socket, this mainly because there will be multiple packets. This is done in the `socket_receive_threaded` function

In [ ]:
print_lock = threading.Lock() 
# thread function 
def socket_receive_threaded(sock, size): 
    BYTES_PER_PACKET = 1408
    shape_global = (size,1)
    shape_local = (BYTES_PER_PACKET,1)
    recv_data_global = np.empty(shape_global, dtype = np.uint8)
    data_partial = np.empty(shape_local, dtype = np.uint8)
    num_it = (size // BYTES_PER_PACKET)
    global mm2s_buf
    sum_bytes = 0
    connection = 'None'
    for m in range(num_it):
        res = sock.recvfrom_into(data_partial) 
        recv_data_global[(m * BYTES_PER_PACKET) : ((m * BYTES_PER_PACKET) \
                        + BYTES_PER_PACKET)] = data_partial
        sum_bytes = sum_bytes + int(res[0])
        connection = res[1]
    msg = "SUCCESS!" if np.array_equal(mm2s_buf, recv_data_global) else "FAILURE!"
    print ("Kernel sending data to the network and the host getting data from network"
    " was a: {}. Total data received {:,} bytes from {}".format(msg,sum_bytes,connection))

* Copy random data to Alveo global memory
* Acquire thread 
* Launch threaded function
* Start kernel memory mapped to stream kernel, this will start sending UDP packets from the Alveo card to the NIC
* Threaded function will start receiving data and it will print out result once ALL data was collected

In [ ]:
mm2s_buf.sync_to_device()
print_lock.acquire() 
start_new_thread(socket_receive_threaded, (sock,size,))
mm2s_wh = mm2s.start(mm2s_buf,size, 1)

## Free resources
Delete buffers and free Alveo card

In [ ]:
del mm2s_buf
del s2mm_buf
del udp_message_global
pynq.Overlay.free(ol)

------------------------------------------
Copyright (c) 2020-2021, Xilinx, Inc.